In [1]:
# ETAPA: DOWNLOAD DE 3 ANOS (OHLCV DIÁRIO) — 24 TICKERS + IBOV
# Objetivo: baixar 3 anos de dados para a carteira base e salvar Parquets + manifesto.

%pip install tqdm
from pathlib import Path
from datetime import datetime, timedelta, timezone
import pandas as pd
from tqdm import tqdm
import yfinance as yf

# Diretório SSOT local
BASE_DIR = Path("/home/wrm/BOLSA_2026/dados_originais")
BASE_DIR.mkdir(parents=True, exist_ok=True)

# Seleção interna balanceada (24)
TICKERS_24 = [
    # Financeiro (4)
    "ITUB4.SA", "BBAS3.SA", "B3SA3.SA", "PSSA3.SA",
    # Materiais / Papel & Celulose / Metais (4)
    "VALE3.SA", "GGBR4.SA", "CSNA3.SA", "SUZB3.SA",
    # Energia / Óleo & Gás / Combustíveis (3)
    "PETR4.SA", "PRIO3.SA", "UGPA3.SA",
    # Utilidades Elétricas (3)
    "ELET3.SA", "TAEE11.SA", "CPLE6.SA",
    # Saneamento (1)
    "SBSP3.SA",
    # Telecom (2)
    "VIVT3.SA", "TIMS3.SA",
    # Saúde (2)
    "RDOR3.SA", "HAPV3.SA",
    # Consumo / Industrial / Tecnologia (4)
    "ABEV3.SA", "WEGE3.SA", "TOTS3.SA", "LREN3.SA",
    # Transporte / Infra (1)
    "RAIL3.SA",
]

# Índice Ibovespa
IBOV_TICKER = "^BVSP"

# Janela temporal (3 anos até hoje, com folga)
agora = datetime.now(timezone.utc).astimezone()
data_fim = agora.date()
data_inicio = data_fim - timedelta(days=365*3 + 10)

# Validações
assert len(TICKERS_24) == 24, f"Esperados 24 tickers, obtidos {len(TICKERS_24)}"
for tk in TICKERS_24:
    if not tk.endswith(".SA"):
        raise ValueError(f"Ticker sem sufixo .SA: {tk}")

def baixar_e_salvar(ticker: str, start_date: str, end_date: str, base_dir: Path) -> dict:
    df = yf.download(
        ticker, start=start_date, end=end_date,
        progress=False, interval="1d", auto_adjust=False, threads=True
    )
    if not df.empty:
        df = (df.rename(columns={
                "Open": "open", "High": "high", "Low": "low",
                "Close": "close", "Adj Close": "adj_close", "Volume": "volume"
             })
             .reset_index()
             .rename(columns={"Date": "date"}))
        df["ticker"] = ticker
        out_path = base_dir / f"{ticker.replace('^','').replace('.','_')}_3y.parquet"
        df.to_parquet(out_path, index=False)
        return {
            "ticker": ticker, "rows": len(df),
            "first_date": df["date"].min().strftime("%Y-%m-%d"),
            "last_date": df["date"].max().strftime("%Y-%m-%d"),
            "file_path": str(out_path), "status": "OK"
        }
    else:
        return {"ticker": ticker, "rows": 0, "first_date": None, "last_date": None,
                "file_path": None, "status": "VAZIO"}

metas = []
for tk in tqdm(TICKERS_24 + [IBOV_TICKER], desc="Baixando 3 anos (OHLCV)"):
    metas.append(baixar_e_salvar(
        ticker=tk,
        start_date=data_inicio.strftime("%Y-%m-%d"),
        end_date=data_fim.strftime("%Y-%m-%d"),
        base_dir=BASE_DIR
    ))

manifesto = pd.DataFrame(metas)
manifesto_path = BASE_DIR / "manifesto_dados_originais_3y.csv"
manifesto.to_csv(manifesto_path, index=False)

print("\nResumo do manifesto:")
print(manifesto[["ticker", "rows", "first_date", "last_date", "status", "file_path"]]
      .to_string(index=False))
print(f"\nArquivos salvos em: {BASE_DIR}")
print(f"Manifesto: {manifesto_path}")


  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)


Note: you may need to restart the kernel to use updated packages.


Baixando 3 anos (OHLCV):   0%|                                       | 0/25 [00:00<?, ?it/s]

Baixando 3 anos (OHLCV):   4%|█▏                             | 1/25 [00:01<00:29,  1.23s/it]

Baixando 3 anos (OHLCV):   8%|██▍                            | 2/25 [00:01<00:19,  1.18it/s]

...

Baixando 3 anos (OHLCV):  92%|███████████████████████████▌  | 23/25 [00:14<00:01,  1.60it/s]

Baixando 3 anos (OHLCV):  96%|████████████████████████████▊ | 24/25 [00:14<00:00,  1.72it/s]

Baixando 3 anos (OHLCV): 100%|██████████████████████████████| 25/25 [00:15<00:00,  1.76it/s]

Baixando 3 anos (OHLCV): 100%|██████████████████████████████| 25/25 [00:15<00:00,  1.62it/s]


Resumo do manifesto:
   ticker  rows first_date  last_date status                                                 file_path
 ITUB4.SA   754 2022-09-06 2025-09-12     OK  /home/wrm/BOLSA_2026/dados_originais/ITUB4_SA_3y.parquet
 BBAS3.SA   754 2022-09-06 2025-09-12     OK  /home/wrm/BOLSA_2026/dados_originais/BBAS3_SA_3y.parquet
 B3SA3.SA   754 2022-09-06 2025-09-12     OK  /home/wrm/BOLSA_2026/dados_originais/B3SA3_SA_3y.parquet
 PSSA3.SA   754 2022-09-06 2025-09-12     OK  /home/wrm/BOLSA_2026/dados_originais/PSSA3_SA_3y.parquet
 VALE3.SA   754 2022-09-06 2025-09-12     OK  /home/wrm/BOLSA_2026/dados_originais/VALE3_SA_3y.parquet
 GGBR4.SA   754 2022-09-06 2025-09-12     OK  /home/wrm/BOLSA_2026/dados_originais/GGBR4_SA_3y.parquet
 CSNA3.SA   754 2022-09-06 2025-09-12     OK  /home/wrm/BOLSA_2026/dados_originais/CSNA3_SA_3y.parquet
 SUZB3.SA   754 2022-09-06 2025-09-12     OK  /home/wrm/BOLSA_2026/dados_originais/SUZB3_SA_3y.parquet
 PETR4.SA   754 2022-09-06 2025-09-12     OK  /home